<a href="https://colab.research.google.com/github/kkrish39/Machine-learning-notebooks/blob/main/apriori_principle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import glob

In [7]:
#folder location of the cgmlunch and boluslunch files
cgmFileArray = sorted(glob.glob('./DataFolder/CGMSeriesLunchPat*.csv'))
bolusFileArray = sorted(glob.glob('./DataFolder/InsulinBolusLunchPat*.csv'))

In [8]:
#running the rule generation for all the files
for i in range(0, len(cgmFileArray)):
    mealNoMealDataFrame = pd.DataFrame()
    mealNoMealDataFrame = pd.DataFrame(pd.read_csv(cgmFileArray[i]))
    insulinBolusLunchData = pd.DataFrame(pd.read_csv(bolusFileArray[i]))
    mealNoMealDataFrameArray = mealNoMealDataFrame.to_numpy()

In [26]:
    # reversing the arrays
    mealNoMealDataFrameArray = mealNoMealDataFrameArray[...,::-1]

In [28]:
    #removing the first column of the array since it holds some NaN values
    mealNoMealDataFrameArray = mealNoMealDataFrameArray[:,1:]
    mealNoMealDataFrame = pd.DataFrame(mealNoMealDataFrameArray)
    

In [11]:
    aprioriData = pd.DataFrame()

In [12]:
    #reading the cgm start time
    aprioriData['CGM_0'] = mealNoMealDataFrame[5]

In [13]:
    #calculating the max glucose level in a row
    aprioriData['CGM_MAX'] = mealNoMealDataFrame.max(axis=1)

In [14]:
    #insulin bolus data
    aprioriData['INSULIN_BOLUS'] = insulinBolusLunchData.max(axis=1)

In [15]:
    cut_labels_4 = list(range(0,37))
    cut_bins = [-1,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300,310,320,330,340,350,360,370,380,390,400]
    aprioriData['CGM_O_BINS'] = pd.cut(aprioriData['CGM_0'], bins=cut_bins, labels=cut_labels_4)
    aprioriData['CGM_MAX_BINS'] = pd.cut(aprioriData['CGM_MAX'], bins=cut_bins, labels=cut_labels_4)

In [16]:
    #generating bins for the CGM_O data
    binArray = aprioriData['CGM_O_BINS'].to_numpy()
    binArray = np.nan_to_num(binArray)
    binArray=binArray.astype(dtype=np.int32)
    aprioriData['CGM_O_BINS'] = binArray

In [17]:
    #generating bins for the CGM_MAX data
    binArray = aprioriData['CGM_MAX_BINS'].to_numpy()
    binArray = np.nan_to_num(binArray)
    binArray=binArray.astype(dtype=np.int32)
    aprioriData['CGM_MAX_BINS'] = binArray

In [18]:
    aprioriData=aprioriData.drop('CGM_0', axis=1)
    aprioriData=aprioriData.drop('CGM_MAX', axis=1)

    aprioriData = list(aprioriData.to_numpy())

In [21]:
    from efficient_apriori import apriori, itemsets_from_transactions, generate_rules_apriori

    itemset, rules = apriori(aprioriData, min_support = 0.01)

In [34]:
    #filter to keep restriction to get the rule in the expected format.
    rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1 
    and list(rule.lhs)[0].is_integer() == True and list(rule.lhs)[1].is_integer() == True and isinstance(rule.rhs[0], float) == True, rules)

    tids, _ = itemsets_from_transactions(aprioriData,min_support = 0.01, output_transaction_ids=True)

    aprioriDataFrame = pd.DataFrame()

In [48]:
    #generated frequent rules
    with open('most_frequent.csv', 'a', newline='') as file:
        for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
            row = list(rule.lhs)
            row.append(list(rule.rhs)[0])
            row.append(rule.confidence)
            row.append(rule.support)
            row = np.array(row)
            aprioriDataFrame = aprioriDataFrame.append(pd.DataFrame([np.array(row)]), ignore_index=True)
            np.savetxt(file,np.asarray(["{"+str(list(rule.lhs)[0])+","+str(list(rule.lhs)[1])+
            ","+str(list(rule.rhs)[0])+"}"]), fmt='%s')
            
    aprioriDataFrame

,0,1,2,3,4
0,14.0,19.0,2.9,1.0,0.055556
1,0.0,35.0,2.9,1.0,0.055556
2,18.0,32.0,2.9,1.0,0.055556
3,7.0,17.0,2.9,1.0,0.055556
4,8.0,28.0,2.9,1.0,0.055556
5,8.0,23.0,2.9,1.0,0.055556
6,10.0,19.0,2.9,1.0,0.055556
7,4.0,30.0,2.9,1.0,0.055556
8,11.0,24.0,2.9,1.0,0.055556
9,10.0,22.0,2.9,1.0,0.055556


In [39]:
    largestConfidence  = aprioriDataFrame.loc[aprioriDataFrame[3] == aprioriDataFrame.max(axis=0)[3]]
    largestConfidence

,0,1,2,3,4
0,14.0,19.0,2.9,1.0,0.055556
1,0.0,35.0,2.9,1.0,0.055556
2,18.0,32.0,2.9,1.0,0.055556
3,7.0,17.0,2.9,1.0,0.055556
4,8.0,28.0,2.9,1.0,0.055556
5,8.0,23.0,2.9,1.0,0.055556
6,10.0,19.0,2.9,1.0,0.055556
7,4.0,30.0,2.9,1.0,0.055556
8,11.0,24.0,2.9,1.0,0.055556
9,10.0,22.0,2.9,1.0,0.055556


In [41]:
    #rules with the highest confidence
    with open('largest_confident.csv', 'a', newline='') as file:
            for index, row in largestConfidence.iterrows():
                np.savetxt(file,np.asarray(["{"+str(row[0])+","+str(row[1])+
                " --> "+str(row[2])+"}"]), fmt='%s')

In [43]:
    anamolousEventFrame  = aprioriDataFrame.loc[aprioriDataFrame[3] == aprioriDataFrame.min(axis=0)[3]]
    anamolousEventFrame

,0,1,2,3,4
0,14.0,19.0,2.9,1.0,0.055556
1,0.0,35.0,2.9,1.0,0.055556
2,18.0,32.0,2.9,1.0,0.055556
3,7.0,17.0,2.9,1.0,0.055556
4,8.0,28.0,2.9,1.0,0.055556
5,8.0,23.0,2.9,1.0,0.055556
6,10.0,19.0,2.9,1.0,0.055556
7,4.0,30.0,2.9,1.0,0.055556
8,11.0,24.0,2.9,1.0,0.055556
9,10.0,22.0,2.9,1.0,0.055556


In [44]:
    #rules with certain anomoly
    with open('anomolous_event.csv', 'a', newline='') as file:
         for index, row in anamolousEventFrame.iterrows():
                np.savetxt(file,np.asarray(["{"+str(row[0])+","+str(row[1])+
                " --> "+str(row[2])+"}"]), fmt='%s')